# Avito Demand Prediction Challenge

Контест Avito по предсказанию успешности объявления, основываясь на его тексте и изображениях.

В рамках курса "Автоматическая обработка естественного языка" контест решали: 
* Дима Татаринов, БКЛ152,
* Даша Максимова, БКЛ151.

## 1. Данные

```
!mkdir data
!gsutil cp -r gs://python-ml-hse/avito/data/data .
```
или

```
!gsutil cp -r gs://python-ml-hse/avito/data/data/train.csv ./data/train.csv
!gsutil cp -r gs://python-ml-hse/avito/data/data/test.csv ./data/test.csv
```

### 1.1. Загрузка

In [1]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 500)

In [2]:
data = pd.read_csv("./data/train.csv")
data.head()

,item_id,user_id,region,city,parent_category_name,category_name,param_1,param_2,param_3,title,description,price,item_seq_number,activation_date,user_type,image,image_top_1,deal_probability
0,b912c3c6a6ad,e00f8ff2eaf9,Свердловская область,Екатеринбург,Личные вещи,Товары для детей и игрушки,Постельные принадлежности,NaN,NaN,Кокоби(кокон для сна),"Кокон для сна малыша,пользовались меньше месяц...",400.0,2,2017-03-28,Private,d10c7e016e03247a3bf2d13348fe959fe6f436c1caf64c...,1008.0,0.12789
1,2dac0150717d,39aeb48f0017,Самарская область,Самара,Для дома и дачи,Мебель и интерьер,Другое,NaN,NaN,Стойка для Одежды,"Стойка для одежды, под вешалки. С бутика.",3000.0,19,2017-03-26,Private,79c9392cc51a9c81c6eb91eceb8e552171db39d7142700...,692.0,0.00000
2,ba83aefab5dc,91e2f88dd6e3,Ростовская область,Ростов-на-Дону,Бытовая электроника,Аудио и видео,"Видео, DVD и Blu-ray плееры",NaN,NaN,Philips bluray,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,9,2017-03-20,Private,b7f250ee3f39e1fedd77c141f273703f4a9be59db4b48a...,3032.0,0.43177
3,02996f1dd2ea,bf5cccea572d,Татарстан,Набережные Челны,Личные вещи,Товары для детей и игрушки,Автомобильные кресла,NaN,NaN,Автокресло,Продам кресло от0-25кг,2200.0,286,2017-03-25,Company,e6ef97e0725637ea84e3d203e82dadb43ed3cc0a1c8413...,796.0,0.80323
4,7c90be56d2ab,ef50846afc0b,Волгоградская область,Волгоград,Транспорт,Автомобили,С пробегом,ВАЗ (LADA),2110,"ВАЗ 2110, 2003",Все вопросы по телефону.,40000.0,3,2017-03-16,Private,54a687a3a0fc1d68aed99bdaaf551c5c70b761b16fd0a2...,2264.0,0.20797


В датасете имеются такие колонки-фичи:

* `item_id` - Ad id.
* `user_id` - User id.
* `region` - Ad region.
* `city` - Ad city.
* `parent_category_name` - Top level ad category as classified by Avito's ad model.
* `category_name` - Fine grain ad category as classified by Avito's ad model.
* `param_1` - Optional parameter from Avito's ad model.
* `param_2` - Optional parameter from Avito's ad model.
* `param_3` - Optional parameter from Avito's ad model.
* `title` - Ad title.
* `description` - Ad description.
* `price` - Ad price.
* `item_seq_number` - Ad sequential number for user.
* `activation_date`- Date ad was placed.
* `user_type` - User type.
* `image` - Id code of image. Ties to a jpg file in train_jpg. Not every ad has an image.
* `image_top_1` - Avito's classification code for the image.
* `deal_probability` - The target variable. This is the likelihood that an ad actually sold something. It's not possible to verify every transaction with certainty, so this column's value can be any float from zero to one.


Мы уберём следующие из них:
* `item_id`, `user_id` — это бесполезная для нас информация,
* `city` — кажется слишком мелким делением,
* `title` — у нас есть более содержательные тексты,
* `param_1`, `param_2`, `param_3` — опциональные и не всегда присутствующие параметры,
* `activation_date`, `item_seq_number` — тоже выглядит бесполезной,
* `image`, `image_top_1` — мы сконцентрируемся на текстах и категориальных переменных, а не на изображениях.

In [3]:
cols_to_drop = ["item_id", "user_id", "city", "param_1", "param_2", "param_3", "title",
    "activation_date", "item_seq_number", "image", "image_top_1"]
data_new = data.drop(labels=cols_to_drop, axis=1)

In [4]:
data_new.head()

,region,parent_category_name,category_name,description,price,user_type,deal_probability
0,Свердловская область,Личные вещи,Товары для детей и игрушки,"Кокон для сна малыша,пользовались меньше месяц...",400.0,Private,0.12789
1,Самарская область,Для дома и дачи,Мебель и интерьер,"Стойка для одежды, под вешалки. С бутика.",3000.0,Private,0.00000
2,Ростовская область,Бытовая электроника,Аудио и видео,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,Private,0.43177
3,Татарстан,Личные вещи,Товары для детей и игрушки,Продам кресло от0-25кг,2200.0,Company,0.80323
4,Волгоградская область,Транспорт,Автомобили,Все вопросы по телефону.,40000.0,Private,0.20797


Категориальные признаки закодируем:

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
parent_category = LabelEncoder()
parent_category.fit(data_new["parent_category_name"])
data_new["parent_category_name"] = parent_category.transform(data_new["parent_category_name"])

In [7]:
category = LabelEncoder()
category.fit(data_new["category_name"])
data_new["category_name"] = category.transform(data_new["category_name"])

In [8]:
user_type = LabelEncoder()
user_type.fit(data_new["user_type"])
data_new["user_type"] = user_type.transform(data_new["user_type"])

In [9]:
region = LabelEncoder()
region.fit(data_new["region"])
data_new["region"] = region.transform(data_new["region"])

Так датасет выглядит сейчас:

In [10]:
data_new.head()

,region,parent_category_name,category_name,description,price,user_type,deal_probability
0,19,4,42,"Кокон для сна малыша,пользовались меньше месяц...",400.0,1,0.12789
1,17,2,22,"Стойка для одежды, под вешалки. С бутика.",3000.0,1,0.00000
2,16,0,2,"В хорошем состоянии, домашний кинотеатр с blu ...",4000.0,1,0.43177
3,21,4,42,Продам кресло от0-25кг,2200.0,0,0.80323
4,4,6,0,Все вопросы по телефону.,40000.0,1,0.20797


### 1.2. Предобработка: тексты

Экспериментальным путём выяснено, что некоторые объявления пустые. Чтобы об них ничего не ломалось, запустим `fillna()`.

In [11]:
data_new["description"].fillna("", inplace=True)

Тексты сначала приведём к нижнему регистру, а затем посчитаем количество токенов и приведём всё к леммам. По пути уберём стоп-слова. Это можно сделать так:

```python
from nltk import word_tokenize
from pymystem3 import Mystem
from nltk.corpus import stopwords

mystem = Mystem()


def count_words(text):
    try:
        len_words = len(word_tokenize(text))
    except:
        len_words = 0
    return len_words

def do_lemmas(text):
    try:
        stops = stopwords.words("russian")
        lemmas = [lemma for lemma in mystem.lemmatize(text) if lemma not in stops]
        return lemmas
    except:
        return ""
        
data_new["word_count"] = data_new["description"].apply(count_words)
data_new["lemmas"] = data_new["description"].apply(do_lemmas)
```

### 1.3. TF-IDF текстов + отделение фич

Для начала выделим целевую переменную и все категориальные и количественные фичи:

In [12]:
cat_num_cols = ["region", "parent_category_name", "category_name", "price", "user_type"]
X_cat_num = data_new[cat_num_cols].values

In [13]:
y = data_new["deal_probability"].values

Теперь применим TF-IDF:

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [19]:
tfidf = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents="unicode",
    analyzer="word",
    token_pattern=r"\w{1,}",
    stop_words=stopwords.words("russian"),
    max_features=10000
)

In [20]:
tfidf.fit(data_new["description"].values)
X_texts = tfidf.transform(data_new["description"].values)

## 2. Объединение фич

In [21]:
from scipy.sparse import hstack

In [22]:
X = hstack(X_texts, X_cat_num)

ValueError: blocks must be 2-D

In [ ]:
X.shape